In [1]:
from collections import Counter
import re

file = ["www.google.com",
        "www.facebook.com",
        "www.youtube.com",
        "www.baidu.com",
        "www.yahoo.com",
        "www.amazon.com",
        "www.wikipedia.org",
        "www.qq.com",
        "www.taobao.com",
        "www.twitter.com",
        "www.google.co.in",
        "www.live.com",
        "www.sina.com.cn",
        "www.linkedin.com",
        "www.weibo.com",
        "www.yahoo.co.jp",
        "www.google.co.jp",
        "www.google.ca",
        "www.ok.ru",
        "www.adcash.com",
        "www.google.com.mx",
        "www.diply.com",
        "www.tianya.cn",
        "www.google.com.hk",
        "www.pornhub.com",
        "www.alibaba.com",
        "www.rakuten.co.jp",
        "www.naver.com",
        "www.amazon.co.uk",
        "www.google.com.tr",
        "www.adobe.com",
        "www.xinhuanet.com",
        "www.gmail.com",
        "www.outbrain.com",
        "www.xhamster.com",
        "www.detail.tmall.com",
        "www.soso.com",
        "www.google.co.id",
        "www.ebay.de",
        "www.kat.cr",
        "www.people.com.cn",
        "www.google.pl",
        "www.jd.com",
        "www.cntv.cn",
        "www.gmw.cn",
        "www.google.com.au",
        "www.go.com",
        "www.nicovideo.jp",
        "www.flipkart.com",
        "www.cnn.com",
        "www.dailymotion.com",
        "www.bbc.co.uk",
        "www.booking.com",
        "www.github.com",
        "www.googleusercontent.com",
        "www.pixnet.net",
        "www.dropbox.com",
        "www.wikia.com",
        "www.163.com",
        "www.chinadaily.com.cn"
        ]

category = {}
category["www.google.com"] = 0
category["www.facebook.com"] = 1
category["www.youtube.com"] = 2
category["www.baidu.com"] = 0
category["www.yahoo.com"] = 1
category["www.amazon.com"] = 2
category["www.wikipedia.org"] = 3
category["www.twitter.com"] = 1
category["www.google.co.in"] = 0
category["www.linkedin.com"] = 1
category["www.yahoo.co.jp"] = 1
category["www.google.co.jp"] = 0
category["www.google.ca"] = 0
category["www.qq.com"] = 2
category["www.taobao.com"] = 2
category["www.live.com"] = 1
category["www.google.com.mx"] = 0
category["www.google.com.hk"] = 0
category["www.pornhub.com"] = 2
category["www.alibaba.com"] = 2
category["www.amazon.co.uk"] = 2
category["www.google.com.tr"] = 0
category["www.adobe.com"] = 0
category["www.gmail.com"] = 0
category["www.xhamster.com"] = 2
category["www.ebay.de"] = 2
category["www.google.pl"] = 0
category["www.google.com.au"] = 0
category["www.dailymotion.com"] = 2
category["www.bbc.co.uk"] = 2
category["www.google.co.id"] = 0
category["www.flipkart.com"] = 2
category["www.cnn.com"] = 2
category["www.booking.com"] = 2
category["www.github.com"] = 0
category["www.dropbox.com"] = 0
category["www.people.com.cn"] = 2
category["www.nicovideo.jp"] = 2
category["www.googleusercontent.com"] = 0
category["www.weibo.com"] = 2
category["www.sina.com.cn"] = 2
category["www.adcash.com"] = 0
category["www.ok.ru"] = 2
category["www.diply.com"] = 2
category["www.tianya.cn"] = 2
category["www.jd.com"] = 2
category["www.cntv.cn"] = 0
category["www.go.com"] = 2
category["www.naver.com"] = 0
category["www.kat.cr"] = 0
category["www.soso.com"] = 0
category["www.wikia.com"] = 0
category["www.163.com"] = 2
category["www.gmw.cn"] = 2
category["www.pixnet.net"] = 3
category["www.outbrain.com"] = 3
category["www.xinhuanet.com"] = 2
category["www.rakuten.co.jp"] = 2
category["www.detail.tmall.com"] = 0
category["www.chinadaily.com.cn"] = 0

def find_PLT(filename):
    file = open(filename)
    list = file.readlines()
    mintime = 0
    for line in list:
        match = re.match('.*requestTime.: ([0-9^.]+)', line)
        if match is not None:
            mintime = float(match.groups(0)[0])
            break
    maxtime = 0
    for line in list:
        match = re.match('.*time.: ([0-9^.]+)',line)
        if match is not None:
            if float(match.groups(0)[0]) > maxtime:
                maxtime = float(match.groups(0)[0])
    
    return (maxtime - mintime,mintime,maxtime)
            
def findcount(text, list):
    feature = Counter()
    for value in list:
        feature[value] = len(re.findall('\.' + value,text))

    return feature


def findAllPLT(name,count):
    list = []
    featureList = ['png','gif','css','js']
    feature = Counter()
    for i in range(count):
        filename = name + str(i)  + ".txt"
        text = open(filename).read()
        feature += findcount(text,featureList)
        (time,min,max) = find_PLT(filename)
        list.append(time)
    
    for key in featureList:
        if key in feature:
            feature[key] = (feature[key]*1.0)/count
        else:
            feature[key] = 0
        
    return feature,list

In [5]:
filepath = "/Users/Aman/Desktop/Colin/data_all/results/"
def all_file():
    dictionary = {}
    for website in file:
        feature,list = findAllPLT(filepath + website,10)
        dictionary[website] = (feature,list)
    return dictionary

all_val_dict = all_file()

import matplotlib.pyplot as plt

for website in file:
    if website in all_val_dict.keys():
        if max(all_val_dict[website][1]) < 8 and min(all_val_dict[website][1])  > 0:
            all_val_dict[website][1].sort()
            all_val_dict[website][1].reverse()
            plt.plot(all_val_dict[website][1],linewidth=2.0)
    
plt.ylabel('Time in seconds ---->')
plt.xlabel('Iterations ---->')
plt.ylim(ymax=8)
plt.grid(True)
plt.title("PAGELOAD TIME  vs  ITERATION")
plt.show()

In [4]:
# import pandas as pd
# import matplotlib.pyplot as plt
# import numpy as np
# from pandas.tools.plotting import parallel_coordinates

# df = pd.DataFrame(columns=('mean','css', 'gif', 'js','png','cat'))
# i = 0
# for website in file:
#     df.loc[i] = [np.mean(all_val_dict[website][1]),all_val_dict[website][0]['css'],all_val_dict[website][0]['gif'],all_val_dict[website][0]['js'],all_val_dict[website][0]['png'],category[website]]
#     i += 1

# df.to_csv('data.csv', sep=',')
# plt.figure()
# parallel_coordinates(df,'cat',colormap='gist_rainbow')
# plt.show()

In [14]:
for website in file:
    plt.scatter(sum(all_val_dict[website][0].values()),np.mean(all_val_dict[website][1]),alpha=0.5)

plt.ylabel('Time in seconds ---->')
plt.xlabel('Number of objects ---->')
plt.grid(True)
plt.title("PAGELOAD TIME  vs  #OBJECTS")
plt.show()



In [17]:
import numpy as np
from sklearn import datasets, linear_model
from scipy import stats as ss

def regression(Y,X):
    X = np.array(X)
    Y = np.array(Y)
    Y = np.array(ss.zscore(Y)).T
    X = np.array([ss.zscore(x) for x in X.T]).T

    regr = linear_model.LinearRegression()
    regr.fit(X,Y)
    print('Coefficients: \n', regr.coef_)
    print("Residual sum of squares: %.2f"% np.mean((regr.predict(X) - Y) ** 2))
    
    return regr    

In [16]:
def getValueRegr(all_val_dict):
    Y = []
    X = []
    for website in file:
        if sum(all_val_dict[website][1]) == 0:
            continue
        Y.append(np.mean(all_val_dict[website][1][0]))
        X.append(all_val_dict[website][0].values())
    
    return (Y,X)

Y,X = getValueRegr(all_val_dict)
print Y
print X

[2.1132289999986824, 2.0249459999995452, 2.9956910000000789, 5.2401100000006409, 3.5522009999995134, 5.687685999999303, 2.1684010000008129, 6.9756170000000566, 6.2029760000004899, 4.5574409999999261, 1.4861350000010134, 1.3221389999998792, 14.220386999999391, 1.8663020000003598, 3.1497500000004948, 7.1314009999996415, 1.5778559999998834, 1.6730129999996279, 5.5951099999983853, 5.3413119999995615, 2.7290460000003804, 4.3440149999987625, 7.356618000000708, 2.8904290000027686, 4.2895239999998012, 5.1689900000001217, 0.0, 3.3258980000009615, 5.0321340000000419, 1.6612199999999575, 8.830257999999958, 11.914213000000018, 3.9389599999994971, 4.4379570000000967, 1.7808799999993425, 9.8002849999993487, 2.2999129999989236, 7.7964319999991858, 2.8425139999999374, 10.022063999999773, 2.1201790000013716, 4.3903610000006665, 1.3529120000002877, 2.2096419999998034, 5.5666220000002795, 4.4613460000000487, 2.4420950000003359, 6.4374109999998836, 9.5774839999994583, 4.8434580000002825, 0.881569999999555

In [19]:
regr = regression(Y,X)
test = all_val_dict['www.facebook.com'][0].values()
test = np.array(ss.zscore(test)).T
print regr.predict(test)

('Coefficients: \n', array([-0.14999042, -0.37004572,  0.16909965,  0.63848421]))
Residual sum of squares: 0.72
[ 0.46429692]


/Users/Aman/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


In [24]:
for website in file:
    sum = 0.0
    i = 0
    for key in all_val_dict[website][0]:
        sum += regr.coef_[i]* all_val_dict[website][0][key]
        i += 1
    plt.scatter(sum,np.mean(all_val_dict[website][1]),alpha=0.5)

plt.ylabel('Time in seconds ---->')
plt.xlabel('Number of objects ---->')
plt.grid(True)
plt.xlim(xmin=-1)
plt.ylim(ymin=0)
plt.title("PAGELOAD TIME  vs  #OBJECTS")
plt.show()


